In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
url = "http://www1.nyc.gov/assets/ccrb/downloads/excel/ccrb_datatransparencyinitiative_20170207.xlsx"

In [5]:
# read data as a pandas dataframe
df =  pd.read_excel(url, sheetname = "Complaints_Allegations")

C:\Users\zhengy\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [6]:
# Do some basic data exploration
df.head(3)

,DateStamp,UniqueComplaintId,Close Year,Received Year,Borough of Occurrence,Is Full Investigation,Complaint Has Video Evidence,Complaint Filed Mode,Complaint Filed Place,Complaint Contains Stop & Frisk Allegations,Incident Location,Incident Year,Encounter Outcome,Reason For Initial Contact,Allegation FADO Type,Allegation Description
0,2017-02-07,6,2006,2006,Brooklyn,False,False,Phone,IAB,False,Street/highway,2006,No Arrest or Summons,PD suspected C/V of violation/crime - street,Discourtesy,Action
1,2017-02-07,11,2006,2006,Bronx,False,False,Phone,IAB,False,Street/highway,2006,Arrest,Other,Force,Physical force
2,2017-02-07,20,2006,2005,Bronx,False,False,Call Processing System,CCRB,True,Street/highway,2005,No Arrest or Summons,Other,Abuse of Authority,Strip-searched


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206718 entries, 0 to 206717
Data columns (total 16 columns):
DateStamp                                      206718 non-null datetime64[ns]
UniqueComplaintId                              206718 non-null int64
Close Year                                     206718 non-null int64
Received Year                                  206718 non-null int64
Borough of Occurrence                          206216 non-null object
Is Full Investigation                          206718 non-null bool
Complaint Has Video Evidence                   206718 non-null bool
Complaint Filed Mode                           206718 non-null object
Complaint Filed Place                          206718 non-null object
Complaint Contains Stop & Frisk Allegations    206718 non-null bool
Incident Location                              203349 non-null object
Incident Year                                  206718 non-null int64
Encounter Outcome                              2067

In [8]:
df.shape

(206718, 16)

In [ ]:
"""
How many unique complaints (identified by 'UniqueComplaintId') with complete information (i.e. there are no missing values) 
appear in the dataset?

"""

In [14]:
df.dropna().UniqueComplaintId.nunique()

68467

In [ ]:
"""
What proportion of complaints occur in the borough with the largest number of complaints? For this question, only consider 
unique complaints with complete information.
"""

In [15]:
# drop incomplete complaints
df_complete = df.dropna()

In [16]:
df_complete.shape

(202390, 16)

In [25]:
# Calulcate the number of unique complaints in each borough
df_complete.groupby("Borough of Occurrence")["UniqueComplaintId"].nunique().sort_values(ascending=False)

Borough of Occurrence
Brooklyn         23369
Manhattan        16067
Bronx            15603
Queens           10608
Staten Island     2763
Outside NYC         57
Name: UniqueComplaintId, dtype: int64

In [36]:
# calculate the total number of unique complaints in NY
df_complete.UniqueComplaintId.nunique()

68467

In [34]:
# Calculate the proportion of complaints occurring in the borough with the largest number of complaints
(df_complete.groupby("Borough of Occurrence")["UniqueComplaintId"].nunique().sort_values(ascending=False)[0])/(df_complete.UniqueComplaintId.nunique())

0.3413177151036266

In [ ]:
"""
How many complaints per 100k residents were there in the borough with the highest number of complaints per capita 
resulting from incidents in 2016? Find the 2016 population estimates of each borough on Wikipedia. Ignore complaints 
from "Outside NYC". For this question, only consider unique complaints with complete information.

"""

In [92]:
# Subset the complaints in 2016
df_2016 = df_complete[df_complete["Incident Year"]==2016]

In [93]:
# Calulcate number of complaints for each borough
df_borough = df_2016.groupby("Borough of Occurrence")["UniqueComplaintId"].nunique()

In [94]:
df_borough.drop(["Outside NYC"], inplace=True)

In [95]:
# Convert to a dataframe
df_borough = df_borough.to_frame()

In [96]:
# Add the population of each borough to the dataframe
df_borough["Population"] = [1455720, 2629150, 1643734, 2333054, 476015]

In [97]:
# Calculate the complaints per captia for each borough
df_borough["complaints_per_capita"] = df_borough["UniqueComplaintId"]/df_borough["Population"]

In [98]:
# Sort the number of complaints per capita
df_borough.sort_values(by=["complaints_per_capita"], ascending=False)

,UniqueComplaintId,Population,complaints_per_capita
Borough of Occurrence,,,
Bronx,772,1455720,0.000530
Manhattan,844,1643734,0.000513
Brooklyn,1068,2629150,0.000406
Staten Island,149,476015,0.000313
Queens,589,2333054,0.000252


In [99]:
# Number of complaints per 100k residents in the borough with the highest number of complaints per capita
df_borough.loc["Bronx", "UniqueComplaintId"]/((df_borough.loc["Bronx", "Population"])/100000)

53.03217651746215

In [ ]:
"""
What is the average number of years it takes for a complaint to be closed? For this question, only consider unique 
complaints with complete information.

"""

In [74]:
df_time = df_complete.drop_duplicates(subset = ["UniqueComplaintId"])

In [76]:
df_time["year"] = df_time["Close Year"] - df_time["Received Year"]

C:\Users\zhengy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [78]:
df_time["year"].mean()

0.4743745161902814

In [ ]:
"""
Complaints about stop and frisk have been declining. Use linear regression from the year complaints about stop and 
frisk peaked through 2016 (inclusive) to predict how many stop and frisk incidents in 2018 will eventually lead to a 
complaint. For this question, only consider unique complaints with complete information. Remember that the count of 
complaints must be an integer (round to nearest).

"""

In [124]:
# Find the complains about stop and flask and set a boolean variable to indicate it
df_time["Frisk"]= (df_time["Allegation Description"].str.contains("Frisk")) | (df_time["Allegation Description"].str.contains("Stop"))

C:\Users\zhengy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [127]:
# Calcualte the number of complaints related to stop and frisk by each year
df_time.groupby("Incident Year")["Frisk"].sum()

Incident Year
1999      0.0
2002      0.0
2003      0.0
2004      6.0
2005     83.0
2006    212.0
2007    613.0
2008    830.0
2009    859.0
2010    727.0
2011    706.0
2012    598.0
2013    499.0
2014    372.0
2015    311.0
2016    231.0
2017      1.0
Name: Frisk, dtype: float64

In [129]:
# Form a dataframe 
df_relationship = pd.DataFrame({"year": [2009, 2010, 2011,2012,2013,2014,2015,2016], "number":[859,727,703,598,499,372,311,231]})

In [130]:
df_relationship

,number,year
0,859,2009
1,727,2010
2,703,2011
3,598,2012
4,499,2013
5,372,2014
6,311,2015
7,231,2016


In [220]:
# Fit the linear regression for the relationship between year and number of complains
import statsmodels.formula.api as sm
model = sm.ols(formula="number ~ year", data=df_relationship).fit()

In [221]:
# Print the fitted model parameters
model.params

Intercept    181854.166667
year            -90.095238
dtype: float64

In [222]:
# Calcuate the number of complaints in 2018 
round(2018* model.params["year"]+model.params["Intercept"])

42.0

In [ ]:
"""
Calculate the chi-square test statistic for testing whether a complaint is more likely to receive a full investigation 
when it has video evidence. For this question, only consider unique complaints with complete information.

"""

In [142]:
# create crosstab for the two columns
pd.crosstab(df_time["Complaint Has Video Evidence"], df_time["Is Full Investigation"], margins = True)

Is Full Investigation,False,True,All
Complaint Has Video Evidence,,,
False,44529,21889,66418
True,584,1465,2049
All,45113,23354,68467


In [137]:
from scipy.stats import chisquare

In [144]:
investigation_video = pd.crosstab(df_time["Complaint Has Video Evidence"], df_time["Is Full Investigation"], margins = True)

In [145]:
# Rename columns and rows
investigation_video.columns = ["no investigation", "investigation", "row_totals"]

investigation_video.index = ["no video","vedio","col_totals"]

In [146]:
investigation_video

,no investigation,investigation,row_totals
no video,44529,21889,66418
vedio,584,1465,2049
col_totals,45113,23354,68467


In [153]:
# Caluclate the expected values
expected =  np.outer(investigation_video["row_totals"][0:2], investigation_video.loc["col_totals"][0:2])/68467

In [155]:
expected = pd.DataFrame(expected)

In [157]:
expected.columns = ["no investigation", "investigation"]
expected.index = ["no video","vedio"]

expected

,no investigation,investigation
no video,43762.911096,22655.088904
vedio,1350.088904,698.911096


In [158]:
observed = investigation_video.iloc[0:2,0:2] 
observed

,no investigation,investigation
no video,44529,21889
vedio,584,1465


In [159]:
# Calculate the chi-square statistics
chi_squared_stat = (((observed-expected)**2)/expected).sum().sum()
chi_squared_stat

1313.7462956601673

In [ ]:
"""
Each row in the data set corresponds with a specific allegation. Therefore a particular complaint, designated by 
'UniqueComplaintId', may have multiple allegations. Consider only allegations with complete information. 
Is the presence of a certain type of allegation (i.e. 'Allegation FADO Type') indicative that a complaint will 
contain multiple allegations? Create indicator variables for whether a complaint contains each type of allegation, 
and perform a linear regression of the number of allegations per complaint against these indicator variables. 
What is the maximum coefficient of the linear regression?

"""

In [173]:
# Subset the data with unique id and allergation type
df_allegation = df[["UniqueComplaintId", "Allegation FADO Type"]]
df_allegation.head()

,UniqueComplaintId,Allegation FADO Type
0,6,Discourtesy
1,11,Force
2,20,Abuse of Authority
3,20,Abuse of Authority
4,20,Abuse of Authority


In [178]:
# Broadcast one column for futher calculating the number of allegations per complaint. 
df_allegation["number"]=1

C:\Users\zhengy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [180]:
# Get indicator variables for whether a complaint contains each type of allegation
df_allegation =pd.get_dummies(df_allegation).groupby("UniqueComplaintId").sum()

In [188]:
# Make the indicator variables to be binary. 
df_allegation["Abuse_of_Authority"] = (df_allegation["Allegation FADO Type_Abuse of Authority"]>0).astype(int)

In [185]:
df_allegation["Discourtesy"] = (df_allegation["Allegation FADO Type_Discourtesy"]>0).astype(int)
df_allegation["Force"] = (df_allegation["Allegation FADO Type_Force"]>0).astype(int)
df_allegation["Language"] = (df_allegation["Allegation FADO Type_Offensive Language"]>0).astype(int)

In [224]:
df_allegation.head()

,number,Allegation FADO Type_Abuse of Authority,Allegation FADO Type_Discourtesy,Allegation FADO Type_Force,Allegation FADO Type_Offensive Language,Abuse of Authority,Discourtesy,Force,Language,Abuse_of_Authority
UniqueComplaintId,,,,,,,,,,
1,5,2,1,2,0,1,1,1,0,1
2,2,2,0,0,0,1,0,0,0,1
3,1,0,1,0,0,0,1,0,0,0
4,3,2,1,0,0,1,1,0,0,1
5,1,0,0,1,0,0,0,1,0,0


In [190]:
# Fit linear regression between indicator variables and number of number of allegation per complaint. 
model2 = sm.ols(formula="number ~ Abuse_of_Authority+Discourtesy+Force+Language", data=df_allegation).fit()

In [193]:
# Get the parameter with the maximum value 
model2.params.sort_values(ascending=False)

Abuse_of_Authority    2.565593
Force                 1.899061
Discourtesy           1.604893
Language              1.470417
Intercept            -0.430303
dtype: float64

In [ ]:
"""
According to NYC.gov there are approximately 36000 officers in New York. The website additionally lists information on 
all the precincts in each borough. Consider unique complaints (not necessarily with complete information) from incidents 
in 2016. Assuming that complaints per capita are proportional to officers per capita in each borough, calculate the average 
number of officers per precinct in each borough (ignore complaints from outside of NYC). What is the ratio of the highest 
number of officers per precinct to the lowest number of officers per precinct?

"""

In [195]:
# Get the incidents happening in 2016
df_2016 = df[df["Incident Year"]==2016]

In [198]:
df_2016.drop_duplicates(subset = ["UniqueComplaintId"], inplace = True)

C:\Users\zhengy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [201]:
# Calculate the number of unique complaints for each borough
incident_per_capita = df_2016.groupby("Borough of Occurrence")["UniqueComplaintId"].count().drop(["Outside NYC"])

In [202]:
incident_per_capita

Borough of Occurrence
Bronx             792
Brooklyn         1102
Manhattan         862
Queens            601
Staten Island     156
Name: UniqueComplaintId, dtype: int64

In [203]:
# Conver to dataframe
incident_per_capita = incident_per_capita.to_frame()

In [204]:
# Add the population to the dataframe
incident_per_capita["population"] = [1455720, 2629150, 1643734, 2333054, 476015]

In [206]:
# Calculate the incident per capita for each borough 
incident_per_capita["incident_per_capita"] = incident_per_capita["UniqueComplaintId"]/incident_per_capita["population"]

In [208]:
# Calculate the ratio of incident per capita for each borough
incident_per_capita["incident_per_capita_ratio"] = incident_per_capita["incident_per_capita"]/(incident_per_capita["incident_per_capita"].sum())

In [212]:
# Calculate the number of officers per borough
incident_per_capita["officer"]=incident_per_capita["incident_per_capita_ratio"]*36000

In [215]:
# Add the number of precints for each borough
incident_per_capita["precinct"] = [12, 23, 22, 16, 4]

In [217]:
# Calculate the officer per precint for each borough 
incident_per_capita["officer_per_precint"]= (incident_per_capita["officer"]/incident_per_capita["precinct"]).round()

In [218]:
incident_per_capita

,UniqueComplaintId,population,incident_per_capita,incident_per_capita_ratio,officer,precinct,officer_per_precint
Borough of Occurrence,,,,,,,
Bronx,792,1455720,0.000544,0.262458,9448.476259,12,787.0
Brooklyn,1102,2629150,0.000419,0.202199,7279.150076,23,316.0
Manhattan,862,1643734,0.000524,0.252981,9107.311072,22,414.0
Queens,601,2333054,0.000258,0.124269,4473.671846,16,280.0
Staten Island,156,476015,0.000328,0.158094,5691.390747,4,1423.0


In [1]:
highest_number = 1423/280
highest_number

5.082142857142857